cd drive/MyDrive/elk-main

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/elk
# !pip install -e .
# !elk elicit microsoft/phi-2 imdb
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive
# !pip install -e .
# !elk elicit microsoft/phi-2 imdb

In [2]:
# prompt: pip install the imports
!pip install -i https://pypi.org/simple/ bitsandbytes --quiet
!pip install transformers --quiet
!pip install datasets --quiet
!pip install accelerate --quiet
!pip install peft --quiet
!pip install tqdm --quiet
!pip install scikit-learn
!pip install matplotlib


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# import os
# from accelerate.utils import write_basic_config

# write_basic_config()  # Write a config file
# os._exit(00)  # Restart the notebook

In [4]:
# torch.cuda.empty_cache()

In [5]:
from tqdm import tqdm
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM #, BitsAndBytesConfig
from sklearn.linear_model import LogisticRegression
import pandas as pd
# import bitsandbytes as bnb
import random

# model_name = "t5"
model_name = "-"



# if you want to cache the model weights somewhere, you can specify that here
cache_dir = None

# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# config = LoraConfig(
#     r=64,
#     lora_alpha=8,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.00,
#     bias="none",
#     task_type="CAUSAL_LM"
# )


if model_name == "deberta":
    model_type = "encoder"
    tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xxlarge", cache_dir=cache_dir)
    model = AutoModelForMaskedLM.from_pretrained("microsoft/deberta-v2-xxlarge", cache_dir=cache_dir)
    model.cuda()
elif model_name == "phiqlora":
    model_type = "decoder"
    model_id = "microsoft/phi-2"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, cache_dir=cache_dir)
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, config).cuda()
elif model_name == "llemaqlora":
    model_type = "decoder"
    model_id = "EleutherAI/llemma_7b"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, cache_dir=cache_dir)
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, config).cuda()
    
# elif model_name == "phi":
#     model_type = "decoder"
#     tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
#     tokenizer.pad_token = tokenizer.eos_token
#     model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
#     model.cuda()
elif model_name == "phi":
    model_type = "decoder"
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
    
    # Define the new special token
    special_token = "[SIGNAL]"
    special_tokens_dict = {'additional_special_tokens': [special_token]}
    pos_token, neu_token, neg_token = '[POS]', '[NEU]', '[NEG]'

    # Add the special token to the tokenizer
    tokenizer.add_special_tokens(special_tokens_dict)
    tokenizer.add_tokens([pos_token, neu_token, neg_token])
    
    # Load the model
    model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
    
    # It's important to resize the token embeddings in the model
    # This adjusts the model to account for the new token(s)
    model.resize_token_embeddings(len(tokenizer))
    
    # Assign the pad token if it's not already set
    tokenizer.pad_token = tokenizer.eos_token
    
    # Make sure your model is on the correct device (e.g., GPU)
    model.cuda()
elif model_name == "phiq":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", cache_dir=cache_dir, load_in_4bit=True)
    # model.cuda()

elif model_name == "phipeft":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
    model = get_peft_model(AutoModelForCausalLM.from_pretrained("microsoft/phi-2", cache_dir=cache_dir), lora_config).cuda()
    model.cuda()

elif model_name == "mistralpeft":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir=cache_dir)
    model = get_peft_model(AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir=cache_dir), lora_config).cuda()

elif model_name == "mistralq":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir=cache_dir, load_in_4bit=True)

elif model_name == "mixtral":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", cache_dir=cache_dir, load_in_4bit=True)
    model.cuda()

elif model_name == "load":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir=cache_dir)
    model = torch.load('mistral_finetune_own_data.ipynb').cuda()

elif model_name == "llemapeft":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/llemma_7b", cache_dir=cache_dir)
    model = get_peft_model(AutoModelForCausalLM.from_pretrained("EleutherAI/llemma_7b", cache_dir=cache_dir, load_in_4bit=True), lora_config).cuda()
elif model_name == "llemma":
    model_type = "decoder"
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/llemma_7b", cache_dir=cache_dir)
    # Define the new special token
    special_token = "[SIGNAL]"
    special_tokens_dict = {'additional_special_tokens': [special_token]}
    
    # Add the special token to the tokenizer
    tokenizer.add_special_tokens(special_tokens_dict)
    model = AutoModelForCausalLM.from_pretrained("EleutherAI/llemma_7b", cache_dir=cache_dir, load_in_4bit=True)
    model.resize_token_embeddings(len(tokenizer))
    
    # Assign the pad token if it's not already set
    tokenizer.pad_token = tokenizer.eos_token
    
    # Make sure your model is on the correct device (e.g., GPU)
    # model.cuda()
else:
    print("Not implemented!")

# peft = True
# if peft:
#     model = get_peft_model(model, lora_config)

Not implemented!


In [6]:
import json

def load_jsonl_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data_point = json.loads(line)
            data.append(data_point)
    return data

file_path = 'phase1_train.jsonl'
data = load_jsonl_data(file_path)

# # Example: Print the first data point to check
# print(json.dumps(data[0], indent=4))
first_completion_text = data[0]['label']['steps'][0]['completions'][0]['text']
# print(first_completion_text)
print(json.dumps(data[7], indent=4))

{
    "labeler": "d8aa7923-b970-45e1-9734-e4a7f6c4a7db",
    "timestamp": "2022-07-17T17:03:28.219211",
    "generation": null,
    "is_quality_control_question": false,
    "is_initial_screening_question": false,
    "question": {
        "problem": "Find the integer $n,$ $-90 < n < 90,$ such that $\\tan n^\\circ = \\tan 312^\\circ.$",
        "ground_truth_answer": "-48"
    },
    "label": {
        "steps": [
            {
                "completions": [
                    {
                        "text": "So we have that $\\tan n^\\circ = \\tan 312^\\circ$.",
                        "rating": 0,
                        "flagged": false
                    },
                    {
                        "text": "So I guess we want to find an integer n such that $\\tan n = \\tan 312$.",
                        "rating": 0,
                        "flagged": false
                    },
                    {
                        "text": "So we're looking for an integer $n$ tha

In [7]:
from itertools import product

def get_all_paths_with_ratings(datapoint, limit=50):
    question = datapoint['question']['problem']

    steps_completions = [step['completions'] for step in datapoint['label']['steps']]


    all_paths_combinations = list(product(*steps_completions))
    random.shuffle(all_paths_combinations)
    if len(all_paths_combinations) > limit:
        all_paths_combinations = all_paths_combinations[:limit]

    # print('len', len(all_paths_combinations))
    all_paths = []
    
    for path in all_paths_combinations:
        # Append ' [SIGNAL] \n ' to the end of each step's text, including the question for the first step
        path_texts = []
        path_texts.append(f"{question} \n {path[0]['text']} [SIGNAL] \n ")
        if path[0]['rating'] != -1:
            for completion in path[1:]:
                text = f"{completion['text']} [SIGNAL] \n "
                if completion['rating'] == -1:
                    path_texts.append(text)
                    break
                
                path_texts.append(text)
        concatenated_path_text = ''.join(path_texts)
        # print('p', concatenated_path_text)
    
        # path_ratings = [completion['rating'] for completion in path]
        # path_dict = [concatenated_path_text, path_ratings]
        # # print('r', path_ratings)
        # all_paths.append(path_dict)
        path_ratings = []
        for completion in path:
            rating = completion['rating']
            if rating == -1:
                path_ratings.append(rating)
                break
            path_ratings.append(rating)        
            # print('r', path_ratings)
        path_dict = [concatenated_path_text, path_ratings]
        # if path_dict not in all_paths and len(path_dict)%2==0:
        if path_dict not in all_paths:
            all_paths.append(path_dict)

    sampled_paths = all_paths if len(all_paths) <= limit else random.sample(all_paths, limit)

    return sampled_paths
    # return (all_paths, all_ratings)

# Assuming 'data' contains your loaded JSONL data
eeer = get_all_paths_with_ratings(data[6])

# # Printing each path and its ratings
# for path, ratings in zip(eeer[0], eeer[1]):
#     print('Path:', path)
#     print('Ratings:', ratings)
#     print('---')

print(len(eeer))
print(eeer)


6
[['One day Max says to Liz, "Out of the 25 people taking either English or French, you and I are the only two taking both.\'\' Liz, being mathematically inclined, responds by pointing out that there are exactly twice as many people in the English class as there are in the French class. How many people are taking English but not French? \n I think we should start by letting the number of people taking French be a variable, say $f$. [SIGNAL] \n And then the number of people taking English is $2f$ because there are twice as many people in the English class as there are in the French class. [SIGNAL] \n That\'s right. Now we can use the fact that there are 25 people taking either English or French. [SIGNAL] \n So the total number of people taking either English or French is $f + 2f = 25$. [SIGNAL] \n ', [1, 1, 1, -1]], ['One day Max says to Liz, "Out of the 25 people taking either English or French, you and I are the only two taking both.\'\' Liz, being mathematically inclined, responds b

In [8]:

def is_datapoint_within_char_limit(datapoint, char_limit=3000):
    if datapoint['label']['finish_reason'] == 'give_up':
        return False

    total_chars = sum(len(completion['text']) for step in datapoint['label']['steps'] for completion in step['completions'])
    # print(total_chars)
    return total_chars < char_limit




paths = []
# for i in range(949):
for i in range(50):

    # print('i', i)
    if is_datapoint_within_char_limit(data[i]):
        paths.extend(get_all_paths_with_ratings(data[i]))



In [9]:

print(len(paths))
# print(len(all_variation_datapoints))
print(len(paths[0]))
print(paths[0])

390
2
['How many seconds are in 7.8 minutes? \n 7.8 minutes is the same as 7 minutes and 0.8 minutes. [SIGNAL] \n Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes. [SIGNAL] \n And since there are 60 seconds in a minute, then there are 60 * 0.8 = 48 seconds in 0.8 minutes. [SIGNAL] \n So, in total, there are 420 + 48 = 468 seconds in 7.8 minutes. [SIGNAL] \n Correct.\n\n# Answer\n\n468 [SIGNAL] \n ', [1, 1, 1, 1, 1]]


In [10]:
print(len(paths))
# print(paths[28])
# for i in range(70):
#     print(paths[i])

390


In [11]:


from torch.nn.utils.rnn import pad_sequence
from accelerate import Accelerator

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from sklearn.metrics import accuracy_score
from itertools import chain
import random
import torch.distributed as dist

# from accelerate import FullyShardedDataParallelPlugin
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(model.dtype)
# accelerator = Accelerator(mixed_precision="no")


# def setup():
#     rank = dist.get_rank()
#     world_size = dist.get_world_size()
#     dist.init_process_group("nccl")
#     torch.cuda.set_device(rank)
#     return rank, world_size

# print('s', setup())

# def cleanup():
#     dist.destroy_process_group()

class MathProblemDataset(Dataset):
    def __init__(self, data, tokenizer, max_token_len=1024, special_token="[SIGNAL]"):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        self.special_token = special_token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
                # text, soln, true_label = data[0], data[2], data[3]

        datapoint = self.data[idx]
    
        # if None not in datapoint[:-1]:
        text = datapoint[0]

        labels = datapoint[1]  # Labels are an array of numbers that are either -1, 0, or 1
        # print(len(labels))
        # print('hh', len(labels))

        # print('tt', text)
        # print('LL', label)
        # labels = [1.0 if label is None else label for label in labels]
        # print('h', len(labels))
        labels = [0 if x is None else x for x in labels]
        labels_tensor = torch.tensor(labels, dtype=torch.float)

        # Calculate how much padding is needed for labels to match max_token_len
        padding_length = 30 - labels_tensor.size(0)
    
        padded_labels = F.pad(labels_tensor, (0, padding_length), 'constant', float('nan'))
        text_with_special_token = text + " " # + " " + self.special_token
        # print(text_with_special_token)
        encoding = self.tokenizer(text_with_special_token, max_length=self.max_token_len, padding='max_length', truncation=True, return_tensors="pt")
        # print(encoding)
        return {
          'input_ids': encoding['input_ids'].flatten(),
          'labels': labels_tensor
        }


def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids = torch.stack(input_ids, dim=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-2.0)  # Use -1.0 or any appropriate padding value

    return {'input_ids': input_ids, 'labels': labels}


class PRM(nn.Module):
    def __init__(self, pre_trained_model = None, tokenizer=None):
        super(PRM, self).__init__()
        self.pre_trained_model = pre_trained_model
        self.tokenizer = tokenizer
        # self.linear = nn.Linear(50296, 1).cuda()
        self.sigmoid = nn.Sigmoid().cuda()
    
    def forward(self, data, attention_mask=None):
        outputs = self.pre_trained_model(data, output_hidden_states=True, attention_mask=attention_mask)
        logits = outputs.logits
        signal_token_id = self.tokenizer.convert_tokens_to_ids('[SIGNAL]')
        batch_size, seq_len, vocab_size = logits.shape

        # Getting both batch indices and position indices
        batch_indices, pos_indices = (data == signal_token_id).nonzero(as_tuple=True)
        
        # Initialize a container to hold the positions for each batch
        batched_positions = [[] for _ in range(batch_size)]
    
        # Populate the container with positions for each batch
        for batch_idx, pos_idx in zip(batch_indices, pos_indices):
            batched_positions[batch_idx.item()].append(pos_idx.item())
    
        # Convert lists to tensors
        batched_positions = [torch.tensor(pos_list, device=data.device) if pos_list else torch.tensor([], device=data.device) 
                             for pos_list in batched_positions]
    
        # print('batched_positions', batched_positions)
                
        # signal_positions = (data == signal_token_id).nonzero(as_tuple=True)[1]
        # print('sig', signal_positions)

                # Initialize a list to store signal logits for each batch
        signal_logits = []
        max_len = 0  # Variable to hold the maximum length of signal positions across batches
        pos_token, neu_token, neg_token = '[POS]', '[NEU]', '[NEG]'

        # Extract logits for signal positions in each batch
        for i in range(batch_size):
            # If there are no positions for this batch (empty tensor), continue to the next batch
            if len(batched_positions[i]) == 0:
                # Optionally handle empty positions case, e.g., with zeros or skip
                continue
            
            # Index into the logits tensor for the current batch and positions
            current_batch_logits = logits[i, batched_positions[i], :]
            current_batch_logits = current_batch_logits[:,[self.tokenizer.convert_tokens_to_ids(pos_token), self.tokenizer.convert_tokens_to_ids(neu_token), self.tokenizer.convert_tokens_to_ids(neg_token)]]
            # Append to the list
            signal_logits.append(current_batch_logits)

        # Assuming signal_logits_list is a list of tensors
        signal_logits = [self.sigmoid(logits) for logits in signal_logits]
        # signal_logits = torch.stack(signal_logits, dim=0)
        max_signals = max(logits.shape[0] for logits in signal_logits if logits.numel() > 0)

        
        # return probs
        # return signal_logits
        # Step 2: Pad each tensor to have 'max_signals' rows
        padded_signal_logits = []
        for logits in signal_logits:
            pad_size = max_signals - logits.shape[0]
            if pad_size > 0:
                # Pad the tensor along the first dimension (number of signals)
                padded_tensor = F.pad(logits, (0, 0, 0, pad_size), "constant", value=-2)  # Pad bottom rows with zeros
            else:
                padded_tensor = logits
            padded_signal_logits.append(padded_tensor)
    
        # Step 3: Stack the padded tensors
        if padded_signal_logits:
            stacked_signal_logits = torch.stack(padded_signal_logits, dim=0)
        else:
            # Handle the case where all tensors might be empty or signal_logits is an empty list
            stacked_signal_logits = torch.zeros(0, max_signals, 3, device=data.device)  # Adjust the 3 if different features
        
        return stacked_signal_logits


# e = entries_with_rating_1[:500] + entries_with_rating_minus_1[:500]
# random.shuffle(e)
# print(all_variation_datapoints)


# dataset = MathProblemDataset(paths, tokenizer)
# # for i in range(10):
# #     print('ll', dataset[i]['labels'])
# dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)  # Adjust batch_size as needed

# # mdel = PRM(model)
# mdel = PRM(model) #.to(device)

# # mdel.load_state_dict(torch.load('phi_first900_under1700_max20.pth'))
# num_gpus = 2
# # mdel = torch.nn.parallel.DataParallel(mdel, device_ids=list(range(num_gpus)), dim=0)

# mdel.train()

# optimizer = AdamW(mdel.parameters(), lr=1e-6)
# criterion = nn.BCELoss()
# epoch_losses = []

# model, optimizer, dataloader = accelerator.prepare(
#     model, optimizer, dataloader
# )

# for epoch in range(2):  # Example: 1 epoch, adjust as necessary
#     epoch_loss = 0.0  # Initialize loss for the epoch
#     num_batches = 0  # Keep track of the number of batches
#     data_loader_with_progress = tqdm(dataloader, desc=f'Epoch {epoch+1}/{30}')
#     # Assuming indices 0 and 1 are for normal labels, 2 for -1, and 3 for -2
#     label_mappings = torch.tensor([
#         [0, 1, 0],  # Index 0: maps to label 0
#         [1, 0, 0],  # Index 1: maps to label 1
#         [0, 0, 1],  # Index 2: maps to label -1
#         [0, 0, 0]   # Index 3: maps to label -2 (padding)
#     ], device=accelerator.device)
#     padding_index = 3  # Assuming -2 corresponds to index 3

#     for batch in data_loader_with_progress:
#         optimizer.zero_grad()

#         input_ids = batch['input_ids'].to(accelerator.device)
#         labels = batch['labels'].to(accelerator.device)
#         # print('labels', labels)

#         # Mapping -2 and -1 to their new indices
#         labels = torch.where(labels == -2, 3, labels)  # Maps -2 to index 3
#         labels = torch.where(labels == -1, 2, labels)  # Maps -1 to index 2

#         labels = labels.long()  # Ensure labels are floating-point

#         labels_mapped = label_mappings[labels]
#         mask = labels != 3  

#         # Directly use model's forward pass which now accepts input_ids
#         predictions = mdel(input_ids)
#         valid_predictions = predictions[mask]
#         valid_labels = labels_mapped[mask].float()

#         # Calculate loss only on valid data
#         loss = criterion(valid_predictions, valid_labels)
#         # loss.backward()
#         accelerator.backward(loss)
#         optimizer.step()
#         epoch_loss += loss.item()  # Accumulate the loss over each batch
#         num_batches += 1
# #         data_loader_with_progress.set_description(f"Epoch {epoch+1}/{30}, Loss: {loss.item():.4f}")


#     average_loss = epoch_loss / num_batches  # Calculate average loss for the epoch
#     epoch_losses.append(average_loss)  # Append the average loss to the list
#     print(f"Epoch {epoch+1}/{2}, Average Loss: {average_loss:.4f}")# dataloader = MathProblemDataset(e, tokenizer)

# torch.save(mdel.state_dict(), 'phi_2epochs_under4000.pth')


# # # Remember to save your model if necessary
# torch.save(mdel.state_dict(), 'll_first900_under1000.pth')


In [12]:
from fairscale.nn.data_parallel import FullyShardedDataParallel as FSDP
def init_distributed_mode():
    dist.init_process_group(backend='nccl')
    torch.cuda.set_device(torch.distributed.get_rank())
    
def training_function():
    init_distributed_mode()


    # tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/llemma_7b", cache_dir=cache_dir)

    # Define the new special token
    special_token = "[SIGNAL]"
    special_tokens_dict = {'additional_special_tokens': [special_token]}
    pos_token, neu_token, neg_token = '[POS]', '[NEU]', '[NEG]'

    # Add the special token to the tokenizer
    tokenizer.add_special_tokens(special_tokens_dict)
    tokenizer.add_tokens([pos_token, neu_token, neg_token])
    
    # Load the model
    # model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained("EleutherAI/llemma_7b", cache_dir=cache_dir)

    
    # It's important to resize the token embeddings in the model
    # This adjusts the model to account for the new token(s)
    model.resize_token_embeddings(len(tokenizer))
    
    # Assign the pad token if it's not already set
    tokenizer.pad_token = tokenizer.eos_token
        
    dataset = MathProblemDataset(paths, tokenizer)
    # for i in range(10):
    #     print('ll', dataset[i]['labels'])
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)  # Adjust batch_size as needed

    # mdel = PRM(model)
    mdel = PRM(model, tokenizer)
    # mdel = accelerator.prepare_model(mdel)


    # mdel.load_state_dict(torch.load('phi_first900_under1700_max20.pth'))
    num_gpus = 2
    # mdel = torch.nn.parallel.DataParallel(mdel, device_ids=list(range(num_gpus)), dim=0)
    mdel=mdel.cuda()
    mdel = FSDP(mdel)
    # mdel.train()

    optimizer = AdamW(mdel.parameters(), lr=1e-6)
    criterion = nn.BCELoss()
    epoch_losses = []


    for epoch in range(2):  # Example: 1 epoch, adjust as necessary
        epoch_loss = 0.0  # Initialize loss for the epoch
        num_batches = 0  # Keep track of the number of batches
        data_loader_with_progress = tqdm(dataloader, desc=f'Epoch {epoch+1}/{30}')
        # Assuming indices 0 and 1 are for normal labels, 2 for -1, and 3 for -2
        label_mappings = torch.tensor([
            [0, 1, 0],  # Index 0: maps to label 0
            [1, 0, 0],  # Index 1: maps to label 1
            [0, 0, 1],  # Index 2: maps to label -1
            [0, 0, 0]   # Index 3: maps to label -2 (padding)
        ], device=device)
        padding_index = 3  # Assuming -2 corresponds to index 3

        for batch in data_loader_with_progress:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            # print('labels', labels)

            # Mapping -2 and -1 to their new indices
            labels = torch.where(labels == -2, 3, labels)  # Maps -2 to index 3
            labels = torch.where(labels == -1, 2, labels)  # Maps -1 to index 2

            labels = labels.long()  # Ensure labels are floating-point

            labels_mapped = label_mappings[labels]
            mask = labels != 3  

            # Directly use model's forward pass which now accepts input_ids
            predictions = mdel(input_ids)
            valid_predictions = predictions[mask]
            valid_labels = labels_mapped[mask].float()

            # Calculate loss only on valid data
            loss = criterion(valid_predictions, valid_labels)
            # loss.backward()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()  # Accumulate the loss over each batch
            num_batches += 1
            data_loader_with_progress.set_description(f"Epoch {epoch+1}/{30}, Loss: {loss.item():.4f}")

    

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function, num_processes=2)

Launching training on 2 GPUs.


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.

In [ ]:
# # torch.save(mdel.state_dict(), 'phi_first900_under1700_max20.pth')
# mdel = CustomPhiQloraModel(model)
# mdel.load_state_dict(torch.load('phi_first900_under1700_max20.pth'))
torch.save(mdel.state_dict(), 'phi_phase1_epoch_1_under3000.pth')


In [ ]:
test_file_path = 'phase1_test.jsonl'
test_data = load_jsonl_data(test_file_path)

test_variations = []
for i in range(5):
    if is_datapoint_within_char_limit(test_data[i], char_limit=3000):
        test_variations.extend(get_all_paths_with_ratings(test_data[i], limit=10000))
        # print(get_all_paths_with_ratings(test_data[i], limit=5))
        # print(test_data[i]['question']['problem'])
        # print('i', i)

print(test_variations[0])
# test_variation_datapoints = process_variations(test_variations)

# print(mdel.forward(dataloader[0].to(device)))
test_dataset = MathProblemDataset(test_variations, tokenizer)
# for i in range(10):
#     print('ll', dataset[i]['labels'])
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Adjust batch_size as needed


epoch_losses = []

mdel.eval()
errors = []
for batch in test_dataloader:
    # optimizer.zero_grad()

    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    labels = labels.float()  # Ensure labels are floating-point
    # print(labels)
    # for label in labels:
    #     label = label != -5000
    # valid_labels_mask = (labelçs != -5000)
    # Apply the mask to filter out -5000 values
    # labels = labels[valid_labels_mask]
    # print(labels)

    # Directly use model's forward pass which now accepts input_ids
    with torch.no_grad():
        predictions = mdel(input_ids)
    # predictions = predictions.clamp(0, 1)
    # labels = labels.clamp(0, 1)

    # print(predictions.shape, labels.shape)
    # print(predictions, labels)
    # # print(predictions)
    print('preds', predictions.squeeze())
    print('labels', labels.squeeze())
    print('----')
    # print(sum(predictions.squeeze()))
    # print(sum(labels.squeeze()))
    # print(sum(labels.squeeze()) - sum(predictions.squeeze()))
    # errors.append(abs(sum(labels.squeeze()) - sum(predictions.squeeze())))


print(sum(errors))






In [ ]:
print(sum(errors)/len(errors))
